### Load Packages and set working directory

In [91]:
#sys.path.append('/p/home/jusers/stephenson1/juwels/toarstats/myenv/lib/python3.11/site-packages')
#!python3 -m pip install git+https://gitlab.jsc.fz-juelich.de/esde/toar-public/toarstats.git --user
#!python -m site --user-base

In [135]:
sys.path.append('/p/home/jusers/stephenson1/juwels/.local/lib/python3.11/site-packages')
import toarstats
import statsmodels.formula.api as smf
from toarstats.trends.interface import calculate_trend
from toarstats.trends.utils import moving_block_bootstrap
from toarstats.trends.quant_reg import quant_reg

In [136]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LinearRegression
from datetime import datetime
import sys
import os
os.chdir('/p/project1/deepacf/stephenson1/ESDP-FinalProject/')
os.getcwd()

'/p/project1/deepacf/stephenson1/ESDP-FinalProject'

In [137]:
## read IAGOS pickle file
data_all = pd.read_pickle("/p/project1/deepacf/stephenson1/ESDP-FinalProject/iagos_combined.pkl")

In [138]:
data_all.head()

,air_press,lon,lat,air_temp,air_temp_flag,RHI,RHI_flag,RHL,RHL_flag,baro_alt
time,,,,,,,,,,
1996-01-01 13:42:04,237.899994,-2.02,55.00,215.410004,13.0,0.995217,10.0,0.548,12.0,10682.000000
1996-01-01 13:42:08,237.800003,-2.02,55.00,215.460007,13.0,0.966160,10.0,0.532,12.0,10682.900391
1996-01-01 13:42:12,237.800003,-2.04,55.02,215.369995,13.0,0.939358,10.0,0.516,12.0,10684.200195
1996-01-01 13:42:16,237.800003,-2.04,55.02,215.119995,13.0,0.915692,10.0,0.503,12.0,10684.200195
1996-01-01 13:42:20,237.800003,-2.06,55.02,214.880005,13.0,0.892026,10.0,0.490,12.0,10684.200195


### User input for variable to be analyzed

In [139]:
## User input
## define data column to be analysed
## 'air_temp', 'RHL', or 'RHI'
analysis_col = 'air_temp'
data_all[analysis_col]

time
1996-01-01 13:42:04    215.410004
1996-01-01 13:42:08    215.460007
1996-01-01 13:42:12    215.369995
1996-01-01 13:42:16    215.119995
1996-01-01 13:42:20    214.880005
                          ...    
2023-01-01 07:36:45    205.250000
2023-01-01 07:36:49    205.250000
2023-01-01 07:36:53    205.309998
2023-01-01 07:36:57    205.330002
2023-01-01 07:37:01    205.350006
Name: air_temp, Length: 116079183, dtype: float64

### Define Functions

In [140]:
## function to return slope of linear regression of pressure vs other varible
def slopePressure(x_col, y_col):
    # perform linear regression on the data
    X = x_col.values.reshape(-1, 1)
    y = y_col.values
    
    model = LinearRegression()
    model.fit(X, y)
    
    # Get the slope
    slope = model.coef_[0]
    intercept = model.intercept_

    return(slope, intercept)

In [141]:
## function to normalize variables relative to average pressure
def normalizeVar(slope, press, variable):
    reference_pressure = press.mean()  # Reference altitude in hPa
    
    # Calculate adjustment factor
    adjustment_factor = slope * (press - reference_pressure)
    
    # Normalize temperature to reference pressure
    normalized_variable = variable - adjustment_factor

    return(normalized_variable)

In [142]:
## Modified version of calculate_seasonal_cycle_function from toarstats package
# Function to calculate seasonal cycle for a specific year
def calculate_seasonal_cycle(year_data):
    """Calculate the seasonal cycle"""
    
    df = pd.DataFrame({
        "value": year_data,
        "month": year_data.index.month
    })
    
    # Fit the OLS model
    seasonal_cycle = smf.ols(
        "value ~ np.sin(2*np.pi*month/12) + np.cos(2*np.pi*month/12) "
        "+ np.sin(2*np.pi*month/6) + np.cos(2*np.pi*month/6)",
        df
    ).fit(method="qr").predict(pd.DataFrame({"month": range(1, 13)}))
    
    return seasonal_cycle

In [ ]:
    if len(year_data) < 12:
        return None  # Not enough data to fit the model

### Take Equal Number of Samples for each month (~ 1 sample every 2 hours)

In [143]:
# Assuming data_all is original DataFrame
data_all['year_month'] = data_all.index.to_period('M')  # Create a new column for year-month

# Define the total number of samples and the number of unique months
total_samples = 100000
unique_months = data_all['year_month'].nunique()
samples_per_month = total_samples // unique_months

# Sample from each month
monthly_samples = []
for month, group in data_all.groupby('year_month'):
    if len(group) >= samples_per_month:
        # Sample the required number of rows
        monthly_samples.append(group.sample(n=samples_per_month, random_state=42))
    else:
        # Take all available rows if less than required
        monthly_samples.append(group)

# Concatenate the samples into a single DataFrame
sample = pd.concat(monthly_samples)

# Check the result
#print(sample.head())
print(sample['year_month'].value_counts())  # Verify the number of samples per month

year_month
2019-05    312
2019-06    312
2019-07    312
2019-08    312
2019-09    312
          ... 
1996-04    312
1996-05    312
1996-06    312
1996-07    312
1996-08    312
Freq: M, Name: count, Length: 320, dtype: int64


In [144]:
print(f"Total number of samples: {sample.shape[0]}")

Total number of samples: 99840


In [101]:
print(sample['year_month'].value_counts())

year_month
2019-05    312
2019-06    312
2019-07    312
2019-08    312
2019-09    312
          ... 
1996-04    312
1996-05    312
1996-06    312
1996-07    312
1996-08    312
Freq: M, Name: count, Length: 320, dtype: int64


In [145]:
# If 'time' is the index, sort by index
sample = sample.sort_index()

# Check the result
print(sample.head())

                      air_press    lon    lat    air_temp  air_temp_flag  \
time                                                                       
1996-01-01 14:05:38  216.300003  -6.25  56.15  213.410004           13.0   
1996-01-01 15:26:46  216.300003 -24.96  59.10  211.860001           13.0   
1996-01-01 17:55:02  216.300003 -59.67  54.21  219.509995           13.0   
1996-01-01 19:43:26  217.500000 -72.45  44.21  217.860001           13.0   
1996-01-02 04:16:01  216.100006 -11.27  51.96  216.240005           13.0   

                          RHI  RHI_flag    RHL  RHL_flag      baro_alt  \
time                                                                     
1996-01-01 14:05:38  0.650412      10.0  0.359      12.0  11287.400391   
1996-01-01 15:26:46  0.378606      10.0  0.205      12.0  11286.400391   
1996-01-01 17:55:02  0.189617      10.0  0.112      12.0  11287.700195   
1996-01-01 19:43:26  0.516619      10.0  0.300      12.0  11249.900391   
1996-01-02 04:16:01  0.

In [103]:
## get slope and intercept
slope_int = slopePressure(sample['air_press'], sample[analysis_col])
slope_int

(0.1080596763771215, 197.01566139793)

### Plot variable of interest vs. pressure with regression line

In [104]:
plt.figure(figsize=(10, 6))
plt.scatter(sample['air_press'], sample[analysis_col], color='b', alpha=0.5)
if analysis_col == 'air_temp':
    plt.title('Air Temperature vs Air Pressure')
    plt.xlabel('Air Pressure (hPa)')
    plt.ylabel('Air Temperature (K)')
if analysis_col == 'RHL':
    plt.title('RHL vs Air Pressure')
    plt.xlabel('Air Pressure (hPa)')
    plt.ylabel('RHL (%)')
if analysis_col == 'RHI':
    plt.title('RHI vs Air Pressure')
    plt.xlabel('Air Pressure (hPa)')
    plt.ylabel('RHI (%)')    
    
plt.grid(True)
plt.tight_layout()
plt.axline(xy1=(0, slope_int[1]), slope=slope_int[0], color='r')
plt.xlim(sample['air_press'].min(), sample['air_press'].max())

# Save the plot to a file
plt.savefig(f'Plots/{analysis_col}_vs_air_press_scatter.png')

# Close the current figure to release memory
plt.close()

In [105]:
mean_air_press = data_all['air_press'].mean()

print("mean_air_press:",mean_air_press)

mean_air_press: 231.05692736634387


### Normalize variable of interest using slope relative to pressure

In [146]:
variable_slope = slopePressure(sample['air_press'], sample[analysis_col])[0]

normalized_variable = normalizeVar(variable_slope, sample['air_press'], sample[analysis_col])
data_normalized = pd.concat([sample, normalized_variable], axis=1)

data_normalized.rename(columns={data_normalized.columns[-1]: f'normalized_{analysis_col}'}, inplace=True)

# Display the first few rows to confirm the change
print(data_normalized.head())

                      air_press    lon    lat    air_temp  air_temp_flag  \
time                                                                       
1996-01-01 14:05:38  216.300003  -6.25  56.15  213.410004           13.0   
1996-01-01 15:26:46  216.300003 -24.96  59.10  211.860001           13.0   
1996-01-01 17:55:02  216.300003 -59.67  54.21  219.509995           13.0   
1996-01-01 19:43:26  217.500000 -72.45  44.21  217.860001           13.0   
1996-01-02 04:16:01  216.100006 -11.27  51.96  216.240005           13.0   

                          RHI  RHI_flag    RHL  RHL_flag      baro_alt  \
time                                                                     
1996-01-01 14:05:38  0.650412      10.0  0.359      12.0  11287.400391   
1996-01-01 15:26:46  0.378606      10.0  0.205      12.0  11286.400391   
1996-01-01 17:55:02  0.189617      10.0  0.112      12.0  11287.700195   
1996-01-01 19:43:26  0.516619      10.0  0.300      12.0  11249.900391   
1996-01-02 04:16:01  0.

In [107]:
## Plot normalized data
plt.figure(figsize=(10, 6))
plt.scatter(data_normalized['air_press'], data_normalized[f'normalized_{analysis_col}'], color='b', alpha=0.5)
if analysis_col == 'air_temp':
    plt.title('Normalized Air Temperature vs Air Pressure')
    plt.xlabel('Air Pressure (hPa)')
    plt.ylabel('Air Temperature (K)')
if analysis_col == 'RHL':
    plt.title('Normalized RHL vs Air Pressure')
    plt.xlabel('Air Pressure (hPa)')
    plt.ylabel('RHL (%)')
if analysis_col == 'RHI':
    plt.title('Normalized RHI vs Air Pressure')
    plt.xlabel('Air Pressure (hPa)')
    plt.ylabel('RHI (%)')    
    
plt.grid(True)
plt.tight_layout()
plt.xlim(data_normalized['air_press'].min(), data_normalized['air_press'].max())

# Save the plot to a file
plt.savefig(f'Plots/normalized_{analysis_col}_vs_air_press_scatter.png')

# Close the current figure to release memory
plt.close()

In [147]:
# Assuming data_normalized is DataFrame
data_normalized['date'] = pd.to_datetime(data_normalized.index)
data_normalized.set_index('date', inplace=True)

# Calculate seasonal cycles for each year
seasonal_cycles = {}

# Loop through each year
# exclude 2023, only contains data from January 1st
for year in data_normalized.index.year.unique():
    if year != 2023:
        year_data = data_normalized[data_normalized.index.year == year]

        seasonal_cycles[year] = calculate_seasonal_cycle(year_data[f'normalized_{analysis_col}'])

# Display seasonal cycles for each year
for year, cycle in seasonal_cycles.items():
    print(f"Seasonal Cycle for {year}:")
    print(cycle)
    print("\n")

Seasonal Cycle for 1996:
0     219.397409
1     219.842179
2     221.067278
3     222.472794
4     223.640885
5     224.488973
6     225.061464
7     225.246194
8     224.763252
9     223.470392
10    221.672802
11    220.082556
dtype: float64


Seasonal Cycle for 1997:
0     219.956059
1     220.392140
2     221.356751
3     222.318141
4     223.000397
5     223.475684
6     223.889933
7     224.150455
8     223.932469
9     223.021102
10    221.642242
11    220.420330
dtype: float64


Seasonal Cycle for 1998:
0     220.339516
1     219.989766
2     220.203646
3     221.189096
4     222.775355
5     224.365427
6     225.268005
7     225.147952
8     224.209397
9     222.969074
10    221.852617
11    220.987220
dtype: float64


Seasonal Cycle for 1999:
0     220.448511
1     220.166200
2     220.045232
3     220.695758
4     222.472007
5     224.848838
6     226.611643
7     226.759540
8     225.302101
9     223.207582
10    221.565745
11    220.767322
dtype: float64


Seasonal Cycle f

In [154]:
## Plot normalized data as time series
plt.figure(figsize=(10, 6))
plt.plot(data_normalized.index, data_normalized[f'normalized_{analysis_col}'], color='b', alpha=0.5)
if analysis_col == 'air_temp':
    plt.title('Normalized Air Temperature Time Series')
    plt.xlabel('Date')
    plt.ylabel('Normalized Air Temperature (K)')
if analysis_col == 'RHL':
    plt.title('Normalized RHL Time Series')
    plt.xlabel('Date')
    plt.ylabel('Normalized RHL (%)')
if analysis_col == 'RHI':
    plt.title('Normalized RHI Time Series')
    plt.xlabel('Date')
    plt.ylabel('Normalized RHI (%)')    
    
plt.grid(True)
plt.tight_layout()
#plt.xlim(data_normalized['air_press'].min(), data_normalized['air_press'].max())

# Save the plot to a file
plt.savefig(f'Plots/normalized_{analysis_col}_time_series.png')

# Close the current figure to release memory
plt.close()

In [109]:
# Calculate anomalies for each year from 1996 to 2023
anomalies_list = []

# Loop through each year
for year in range(1996, 2023):
    year_data = data_normalized[data_normalized.index.year == year]
    
    # Check for enough data points
    seasonal_cycle = calculate_seasonal_cycle(year_data[f'normalized_{analysis_col}'])

    idx = year_data.index
    anomalies = year_data[f'normalized_{analysis_col}'].values - seasonal_cycle[idx.month - 1]
    anomalies_list.append(anomalies)
    

anomalies_concat = pd.concat(anomalies_list)

In [110]:
anomalies_concat.shape
anomalies_concat.head()

date
0   -4.532321
0   -6.082324
0    1.567670
0   -0.211995
0   -1.680708
dtype: float64

In [111]:
## append anomalies colulmn to data
data_normalized = data_normalized[data_normalized.index.year != 2023]
data_normalized.shape
anomalies_concat.index = data_normalized.index

data_normalized_anomalies = pd.concat([data_normalized, anomalies_concat], axis=1)

data_normalized_anomalies.rename(columns={data_normalized_anomalies.columns[-1]: f'{analysis_col}_anomalies'}, inplace=True)
data_normalized_anomalies

,air_press,lon,lat,air_temp,air_temp_flag,RHI,RHI_flag,RHL,RHL_flag,baro_alt,year_month,normalized_air_temp,air_temp_anomalies
date,,,,,,,,,,,,,
1996-01-01 14:05:38,216.300003,-6.2500,56.1500,213.410004,13.0,0.650412,10.0,0.359,12.0,11287.400391,1996-01,214.865087,-4.532321
1996-01-01 15:26:46,216.300003,-24.9600,59.1000,211.860001,13.0,0.378606,10.0,0.205,12.0,11286.400391,1996-01,213.315084,-6.082324
1996-01-01 17:55:02,216.300003,-59.6700,54.2100,219.509995,13.0,0.189617,10.0,0.112,12.0,11287.700195,1996-01,220.965078,1.567670
1996-01-01 19:43:26,217.500000,-72.4500,44.2100,217.860001,13.0,0.516619,10.0,0.300,12.0,11249.900391,1996-01,219.185413,-0.211995
1996-01-02 04:16:01,216.100006,-11.2700,51.9600,216.240005,13.0,0.112501,10.0,0.063,12.0,11288.299805,1996-01,217.716701,-1.680708
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-31 16:06:56,227.600006,-33.4589,46.7786,218.720001,20.0,0.152047,20.0,0.089,20.0,10963.000000,2022-12,218.954010,0.913242
2022-12-31 18:31:28,206.600006,-53.1314,36.0211,212.669998,20.0,0.720932,20.0,0.398,20.0,11578.700195,2022-12,215.173261,-2.867508
2022-12-31 19:54:24,206.600006,-62.4719,28.2867,221.979996,20.0,0.134053,20.0,0.081,20.0,11577.500000,2022-12,224.483258,6.442490


In [155]:
## Plot anomalies time series
plt.figure(figsize=(10, 6))
plt.plot(data_normalized_anomalies.index, data_normalized_anomalies[f'{analysis_col}_anomalies'], color='b', alpha=0.5)
if analysis_col == 'air_temp':
    plt.title('Air Temperature Anomalies Time Series')
    plt.xlabel('Date')
    plt.ylabel('Anomaly Air Temperature (K)')
if analysis_col == 'RHL':
    plt.title('RHL Anomalies Time Series')
    plt.xlabel('Date')
    plt.ylabel('Anomaly RHL (%)')
if analysis_col == 'RHI':
    plt.title('RHI Anomalies Time Series')
    plt.xlabel('Date')
    plt.ylabel('Anomaly RHI (%)')    
    
plt.grid(True)
plt.tight_layout()
#plt.xlim(data_normalized['air_press'].min(), data_normalized['air_press'].max())

# Save the plot to a file
plt.savefig(f'Plots/anomalies_{analysis_col}_time_series.png')

# Close the current figure to release memory
plt.close()

In [ ]:
## prep data for quantile regression
data_normalized_anomalies['datetime'] = data_normalized_anomalies.index
data_normalized_anomalies['datetime'] = pd.to_datetime(data_normalized_anomalies['datetime'])


## select anamoly variable of interest and date time column
data_anomalies = data_normalized_anomalies[[f'{analysis_col}_anomalies', 'datetime']]

## rename columns
data_anomalies.rename(columns = {f'{analysis_col}_anomalies' : 'value'}, inplace = True)

## use seconds since first observation as datatime value
data_anomalies['sec_since_start'] = data_anomalies['datetime'] - data_anomalies['datetime'][0]
data_anomalies['sec_since_start'][1].total_seconds()

data_anomalies['secs'] = data_anomalies['sec_since_start'].dt.total_seconds()
data_anomalies = data_anomalies[['value', 'secs']]
data_anomalies.rename(columns = {'secs' : 'datetime'}, inplace = True)
data_anomalies

#data_anomalies['sec_since_start'] = datetime.timedelta(data_anomalies['sec_since_start']).total_seconds()
#temp_anomalies['datetime'] = data_normalized_anomalies.index


In [170]:
quant_model_95 = quant_reg(data_anomalies, .95, num_samples=1000)
quant_model_05 = quant_reg(data_anomalies, .05, num_samples=1000)
quant_model_50 = quant_reg(data_anomalies, .5, num_samples=1000)

In [171]:
print(quant_model_95)
print(quant_model_05)
print(quant_model_50)

{'trend': Intercept   -8.297521e-20
datetime    -4.509150e-11
dtype: float64, 'uncertainty': array([1.48253801e-19, 8.11813055e-11]), 'p_value': array([0.5756966 , 0.57859364])}


In [164]:
np.quantile(data_anomalies['value'], .95)

9.272825848518492

In [167]:
# Plotting
plt.figure(figsize=(10, 6))
plt.plot(data_anomalies['datetime'], data_anomalies['value'], label='Air Temperature Anomalies', alpha=0.5)
plt.axline(xy1=(0, quant_model_95['trend'][0] + np.quantile(data_anomalies['value'], .95)), slope=quant_model_95['trend'][1], color='orange', label='95th Quantile')
plt.axline(xy1=(0, quant_model_05['trend'][0] + np.quantile(data_anomalies['value'], .05)), slope=quant_model_05['trend'][1], color='yellow', label='5th Quantile')
plt.axline(xy1=(0, quant_model_50['trend'][0] + np.quantile(data_anomalies['value'], .5)), slope=quant_model_50['trend'][1], color='red', label='50th Quantile')
#plt.ylim(-5, 5)
plt.xlabel('Seconds since 1996-01-01 14:05:38')
plt.ylabel('Anomaly Air Temperature (K)')
plt.title('Quantile Regression Results Air Temperature')
plt.legend()
plt.savefig('Plots/Quantile_Regression_Results.png')